In [ ]:
import cv2
import os
import numpy as np
from tqdm import tqdm
from PIL import Image

# 원본 이미지 폴더 경로 (변환할 이미지가 들어 있는 최상위 폴더)
input_folder = r"C:\Users\USER\Desktop\1차 프로젝트\train 이미지\200장씩\TS6_selsect"

# 변환된 이미지 저장 폴더 경로 (출력 폴더)
output_folder = r"C:\Users\USER\Desktop\1차 프로젝트\train 이미지\200장씩\openCV\ts6_200"

# 출력 폴더가 없으면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 지원하는 이미지 확장자
valid_extensions = (".jpg", ".jpeg", ".png")

# 하위 폴더까지 포함하여 변환
for root, dirs, files in os.walk(input_folder):
    for filename in tqdm(files, desc=f"Processing {root}", leave=False):
        # 파일 확장자 확인 (폴더는 건너뜀)
        if not filename.lower().endswith(valid_extensions):
            continue

        # 원본 파일 경로
        input_path = os.path.join(root, filename)

        # 변환된 파일의 저장 위치를 원본 폴더 구조와 동일하게 유지
        relative_path = os.path.relpath(root, input_folder)
        output_subfolder = os.path.join(output_folder, relative_path)
        
        # 폴더 생성 확인
        if not os.path.exists(output_subfolder):
            os.makedirs(output_subfolder, exist_ok=True)

        output_path = os.path.join(output_subfolder, filename)

        # 파일 손상 여부 확인
        try:
            with Image.open(input_path) as img:
                img.verify()
        except Exception as e:
            continue

        # PIL을 사용하여 RGB 변환 후 numpy 배열로 변환
        try:
            img = Image.open(input_path).convert("RGB")
            img = np.array(img)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        except Exception as e:
            continue

        # Grayscale 변환 후 히스토그램 평활화
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        equalized = cv2.equalizeHist(gray)

        # Canny Edge Detection 적용
        edges = cv2.Canny(equalized, threshold1=30, threshold2=100)

        # 적외선 효과 적용 (COLORMAP_JET 사용)
        thermal = cv2.applyColorMap(equalized, cv2.COLORMAP_JET)

        # 엣지 강조: 열화상 이미지 + 윤곽선 합성
        thermal[edges > 0] = [255, 255, 255]

        # OpenCV로 이미지 저장
        success = cv2.imwrite(output_path, thermal)

        # OpenCV로 저장 실패 시 Pillow로 저장 시도
        if not success:
            try:
                pil_img = Image.fromarray(cv2.cvtColor(thermal, cv2.COLOR_BGR2RGB))
                pil_img.save(output_path)
            except Exception as e:
                print(f" PIL을 사용한 저장 실패: {output_path}, 오류: {e}")

print(f"\n 모든 이미지 변환 완료! 변환된 이미지가 '{output_folder}'에 저장되었습니다.")